In [344]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import DBSCAN
import warnings
warnings.filterwarnings("ignore")

awards = pd.read_csv("data/awards_data.csv")
player_data = pd.read_csv("data/player_stats.csv")
team_data = pd.read_csv("data/team_stats.csv")
rebounding_data = pd.read_csv("data/team_rebounding_data_22.csv")

# Data Wrangling

## player_data part

**To explain each variables in player_Data file**


'nbapersonid': Unique identifier for each NBA player.

'player': Name of the NBA player.

'draftyear': The year the player was drafted into the NBA.

draftpick: The player's draft pick number in the year they were drafted.

season: The NBA season in which the statistics were recorded.

nbateamid: Unique identifier for each NBA team.

team: Name of the NBA team for which the player played during the specified season.

games: Number of games the player participated in during the season.

games_start: Number of games the player as the started member during the season.

mins: Total minutes played by the player during the season.

fgm: Number of field goals made by the player.

fga: Number of field goals attempted by the player.

fgp: Field goal percentage, calculated as (fgm / fga) * 100.

fgm3: Number of three-point field goals made by the player.

fga3: Number of three-point field goals attempted by the player.

fgp3: Three-point field goal percentage, calculated as (fgm3 / fga3) * 100.

fgm2: Number of two-point field goals made by the player.

fga2: Number of two-point field goals attempted by the player.

fgp2: Two-point field goal percentage, calculated as (fgm2 / fga2) * 100.

efg: Effective field goal percentage, a measure that adjusts for the value of three-point shots, calculated as ((fgm + (0.5 * fgm3)) / fga) * 100.

ftm: Number of free throws made by the player.

fta: Number of free throws attempted by the player.

ftp: Free throw percentage, calculated as (ftm / fta) * 100.

off_reb: Number of offensive rebounds grabbed by the player.

def_reb: Number of defensive rebounds grabbed by the player.

tot_reb: Total number of rebounds grabbed by the player.

ast: Number of assists made by the player.

steals: Number of steals made by the player.

blocks: Number of blocks made by the player.

tov: Number of turnovers committed by the player.

tot_fouls: Total number of fouls committed by the player.

points: Total points scored by the player.

PER: Player Efficiency Rating, a measure of a player's per-minute productivity.

FTr: Free Throw Rate, a ratio of free throws attempted to field goals attempted.

off_reb_pct: Offensive Rebound Percentage, the percentage of available offensive rebounds grabbed by the player while on the court.

def_reb_pct: Defensive Rebound Percentage, the percentage of available defensive rebounds grabbed by the player while on the court.

tot_reb_pct: Total Rebound Percentage, the percentage of available rebounds grabbed by the player while on the court.

ast_pct: Assist Percentage, the percentage of teammate field goals a player assisted while on the court.

stl_pct: Steal Percentage, the percentage of opponent possessions that end with a steal by the player while on the court.

blk_pct: Block Percentage, the percentage of opponent two-point field goal attempts blocked by the player while on the court.

tov_pct: Turnover Percentage, the percentage of the player's possessions that end with a turnover while on the court.

usg: Usage Percentage, an estimate of the percentage of team plays used by a player while on the court.

OWS: Offensive Win Shares, an estimate of the number of wins contributed by a player's offense.

DWS: Defensive Win Shares, an estimate of the number of wins contributed by a player's defense.

WS: Win Shares, an estimate of the total number of wins contributed by a player.

OBPM: Offensive Box Plus/Minus, a box score-based metric that estimates a player's impact on team performance per 100 possessions through offensive contributions.

DBPM: Defensive Box Plus/Minus, a box score-based metric that estimates a player's impact on team performance per 100 possessions through defensive contributions.

BPM: Box Plus/Minus, a box score-based metric that estimates a player's overall impact on team performance per 100 possessions.

VORP: Value over Replacement Player, an estimate of the points per 100 team possessions that a player contributed above a replacement-level player, translated to an average team.

First of all, we will inspect the data.

In [345]:
player_data.head()

,nbapersonid,player,draftyear,draftpick,season,nbateamid,team,games,games_start,mins,...,blk_pct,tov_pct,usg,OWS,DWS,WS,OBPM,DBPM,BPM,VORP
0,2585,Zaza Pachulia,2003,42.0,2007,1610612737,ATL,62,5,944,...,0.010,0.181,0.183,0.2,0.9,1.1,-3.9,-1.3,-5.1,-0.7
1,200780,Solomon Jones,2006,33.0,2007,1610612737,ATL,35,0,145,...,0.026,0.221,0.156,-0.1,0.1,0.0,-6.7,-2.0,-8.8,-0.2
2,2746,Josh Smith,2004,17.0,2007,1610612737,ATL,81,81,2873,...,0.059,0.155,0.250,1.2,4.6,5.8,0.5,2.5,3.0,3.7
3,201151,Acie Law,2007,11.0,2007,1610612737,ATL,56,6,865,...,0.000,0.178,0.165,-0.5,0.4,-0.1,-4.2,-1.0,-5.2,-0.7
4,101136,Salim Stoudamire,2005,31.0,2007,1610612737,ATL,35,0,402,...,0.009,0.094,0.252,0.1,0.1,0.3,-1.0,-2.5,-3.5,-0.1


Then let's understand its structure and dectct any missing values.

In [346]:
print(player_data.isnull().sum())
player_data.dtypes

nbapersonid       0
player            0
draftyear         0
draftpick      1763
season            0
nbateamid         0
team              0
games             0
games_start       0
mins              0
fgm               0
fga               0
fgp              54
fgm3              0
fga3              0
fgp3           1044
fgm2              0
fga2              0
fgp2            115
efg              54
ftm               0
fta               0
ftp             490
off_reb           0
def_reb           0
tot_reb           0
ast               0
steals            0
blocks            0
tov               0
tot_fouls         0
points            0
PER               0
FTr              54
off_reb_pct       0
def_reb_pct       0
tot_reb_pct       0
ast_pct           0
stl_pct           0
blk_pct           0
tov_pct          45
usg               0
OWS               0
DWS               0
WS                0
OBPM              0
DBPM              0
BPM               0
VORP              0
dtype: int64


nbapersonid      int64
player          object
draftyear        int64
draftpick      float64
season           int64
nbateamid        int64
team            object
games            int64
games_start      int64
mins             int64
fgm              int64
fga              int64
fgp            float64
fgm3             int64
fga3             int64
fgp3           float64
fgm2             int64
fga2             int64
fgp2           float64
efg            float64
ftm              int64
fta              int64
ftp            float64
off_reb          int64
def_reb          int64
tot_reb          int64
ast              int64
steals           int64
blocks           int64
tov              int64
tot_fouls        int64
points           int64
PER            float64
FTr            float64
off_reb_pct    float64
def_reb_pct    float64
tot_reb_pct    float64
ast_pct        float64
stl_pct        float64
blk_pct        float64
tov_pct        float64
usg            float64
OWS            float64
DWS        

In [347]:
print(player_data.columns)

Index(['nbapersonid', 'player', 'draftyear', 'draftpick', 'season',
       'nbateamid', 'team', 'games', 'games_start', 'mins', 'fgm', 'fga',
       'fgp', 'fgm3', 'fga3', 'fgp3', 'fgm2', 'fga2', 'fgp2', 'efg', 'ftm',
       'fta', 'ftp', 'off_reb', 'def_reb', 'tot_reb', 'ast', 'steals',
       'blocks', 'tov', 'tot_fouls', 'points', 'PER', 'FTr', 'off_reb_pct',
       'def_reb_pct', 'tot_reb_pct', 'ast_pct', 'stl_pct', 'blk_pct',
       'tov_pct', 'usg', 'OWS', 'DWS', 'WS', 'OBPM', 'DBPM', 'BPM', 'VORP'],
      dtype='object')


Then let's clean the players data by drop and clean the null value

In [348]:
null_values_summary = player_data.isnull().sum()
columns_with_nulls = null_values_summary[null_values_summary > 0]
print(columns_with_nulls)


draftpick    1763
fgp            54
fgp3         1044
fgp2          115
efg            54
ftp           490
FTr            54
tov_pct        45
dtype: int64


Understanding the content, draftpick is not the central to our anlaysis so that we will consider to drop the  variables. the null value in fgp, fgp3, fgp2,efg, ftp, Ftr,tov_pct would be replaced with zeros.This accurately reflects that they had no impact in these specific areas, which is valuable information for your analysis.

In [349]:
player_data_clean = player_data.drop(columns=['draftpick'])
columns_to_replace_zeros = ['fgp', 'fgp3', 'fgp2', 'efg', 'ftp', 'FTr','tov_pct']
player_data_clean[columns_to_replace_zeros] = player_data_clean[columns_to_replace_zeros].replace(np.nan,0)

Then we can detect the duplicate rows

In [350]:
duplicates = player_data_clean.duplicated().sum()
print(f"Number of duplicate rows: {duplicates}")


Number of duplicate rows: 11


In [351]:
duplicated_rows = player_data_clean[player_data_clean.duplicated()]
duplicated_rows


,nbapersonid,player,draftyear,season,nbateamid,team,games,games_start,mins,fgm,...,blk_pct,tov_pct,usg,OWS,DWS,WS,OBPM,DBPM,BPM,VORP
2729,203099,Jared Cunningham,2012,2012,1610612742,DAL,8,0,26,6,...,0.000,0.118,0.296,0.0,0.0,0.0,0.4,-1.9,-1.5,0.0
3140,203099,Jared Cunningham,2012,2013,1610612737,ATL,5,0,22,1,...,0.000,0.410,0.101,-0.1,0.0,-0.1,-9.0,-2.4,-11.4,-0.1
3608,203099,Jared Cunningham,2012,2013,1610612758,SAC,8,0,58,5,...,0.000,0.000,0.191,0.1,0.0,0.2,-3.7,2.4,-1.3,0.0
3911,203099,Jared Cunningham,2012,2014,1610612746,LAC,19,0,89,12,...,0.000,0.153,0.232,-0.1,0.1,0.0,-6.1,-2.0,-8.1,-0.1
4358,203099,Jared Cunningham,2012,2015,1610612739,CLE,40,3,355,32,...,0.005,0.138,0.166,-0.2,0.4,0.1,-5.5,0.0,-5.6,-0.3
4556,203099,Jared Cunningham,2012,2015,1610612749,MIL,4,0,55,4,...,0.000,0.149,0.164,-0.1,0.0,0.0,-5.3,-1.5,-6.8,-0.1
5658,1628476,Derrick Walton Jr.,2017,2017,1610612748,MIA,16,0,147,8,...,0.017,0.067,0.091,0.2,0.2,0.4,-1.1,2.4,1.3,0.1
6722,1628476,Derrick Walton Jr.,2017,2019,1610612765,DET,3,0,26,1,...,0.000,0.250,0.067,0.0,0.0,0.1,-2.9,4.7,1.8,0.0
6804,1628476,Derrick Walton Jr.,2017,2019,1610612746,LAC,23,1,222,17,...,0.000,0.091,0.083,0.4,0.2,0.5,-2.2,0.9,-1.3,0.0
7343,1629686,Deividas Sirvydis,2019,2020,1610612765,DET,20,0,133,14,...,0.000,0.105,0.157,-0.1,0.1,0.0,-4.5,-1.1,-5.6,-0.1


11 duplicateds rows might indicate that the players has participate in different team and games for some other reasons. This is an important observation.

In [352]:
# Convert all names to lower case and all team name to upper case
player_data_clean['player'] = player_data_clean['player'].str.lower()
player_data_clean['team'] = player_data_clean['team'].str.upper()
# convert some features name to a more readable form
player_data_clean = player_data_clean.rename(columns={'nbapersonid': 'personID'})
player_data_clean = player_data_clean.rename(columns={'player': 'name'})
player_data_clean = player_data_clean.rename(columns={'nbateamid': 'teamID'})
player_data_clean = player_data_clean.rename(columns={'games': 'game_played'})
player_data_clean = player_data_clean.rename(columns={'games_start': 'games_as_the_start'})# the number of games that the player played as the starting team

In [353]:
player_data_clean

,personID,name,draftyear,season,teamID,team,game_played,games_as_the_start,mins,fgm,...,blk_pct,tov_pct,usg,OWS,DWS,WS,OBPM,DBPM,BPM,VORP
0,2585,zaza pachulia,2003,2007,1610612737,ATL,62,5,944,107,...,0.010,0.181,0.183,0.2,0.9,1.1,-3.9,-1.3,-5.1,-0.7
1,200780,solomon jones,2006,2007,1610612737,ATL,35,0,145,12,...,0.026,0.221,0.156,-0.1,0.1,0.0,-6.7,-2.0,-8.8,-0.2
2,2746,josh smith,2004,2007,1610612737,ATL,81,81,2873,518,...,0.059,0.155,0.250,1.2,4.6,5.8,0.5,2.5,3.0,3.7
3,201151,acie law,2007,2007,1610612737,ATL,56,6,865,95,...,0.000,0.178,0.165,-0.5,0.4,-0.1,-4.2,-1.0,-5.2,-0.7
4,101136,salim stoudamire,2005,2007,1610612737,ATL,35,0,402,65,...,0.009,0.094,0.252,0.1,0.1,0.3,-1.0,-2.5,-3.5,-0.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8487,1630648,jordan schakel,2021,2021,1610612764,WAS,4,0,30,1,...,0.000,0.078,0.191,-0.2,0.0,-0.1,-8.6,-4.4,-13.0,-0.1
8488,1630557,corey kispert,2021,2021,1610612764,WAS,77,36,1801,234,...,0.010,0.085,0.146,1.6,0.7,2.3,-0.8,-1.5,-2.3,-0.1
8489,1628398,kyle kuzma,2017,2021,1610612764,WAS,66,66,2204,423,...,0.022,0.141,0.242,0.0,2.0,2.0,0.2,-0.4,-0.2,1.0
8490,203526,raul neto,2013,2021,1610612764,WAS,70,19,1372,204,...,0.002,0.139,0.184,0.7,0.8,1.5,-2.5,-0.5,-3.0,-0.4


The duplicated rows are the players who played in different team, which should be evaluated differently. 

## awards parts

Next step we will wrangling awards

In [354]:
awards.columns

Index(['season', 'nbapersonid', 'All NBA Defensive First Team',
       'All NBA Defensive Second Team', 'All NBA First Team',
       'All NBA Second Team', 'All NBA Third Team', 'All Rookie First Team',
       'All Rookie Second Team', 'Bill Russell NBA Finals MVP',
       'Player Of The Month', 'Player Of The Week', 'Rookie Of The Month',
       'all_star_game', 'rookie_all_star_game', 'allstar_rk',
       'Defensive Player Of The Year_rk', 'Most Improved Player_rk',
       'Most Valuable Player_rk', 'Rookie Of The Year_rk',
       'Sixth Man Of The Year_rk', 'all_nba_points_rk',
       'all_rookie_points_rk'],
      dtype='object')

season: The NBA season year for which the awards were given.

nbapersonid: A unique identifier for each NBA player.

All NBA Defensive First Team: The player was selected for the first team of the best defensive players.

All NBA Defensive Second Team: The player was selected for the second team of the best defensive players.

All NBA First Team: The player was selected for the first team of the best players overall.

All NBA Second Team: The player was selected for the second team of the best players overall.

All NBA Third Team: The player was selected for the third team of the best players overall.

All Rookie First Team: The player was selected for the first team of the best rookie players.

All Rookie Second Team: The player was selected for the second team of the best rookie players.

Bill Russell NBA Finals MVP: The player was awarded the Most Valuable Player award for the NBA Finals.

Player Of The Month: The player was named Player of the Month.

Player Of The Week: The player was named Player of the Week.

Rookie Of The Month: The player was named Rookie of the Month.

all_star_game: The player was selected to participate in the All-Star Game.

rookie_all_star_game: The player was selected to participate in the All-Star Game specifically for rookies.

allstar_rk: The player's ranking or points in All-Star Game selections.

Defensive Player Of The Year_rk: The player's ranking or points in the voting for the Defensive Player of the Year award.

Most Improved Player_rk: The player's ranking or points in the voting for the Most Improved Player award.

Most Valuable Player_rk: The player's ranking or points in the voting for the Most Valuable Player award.

Rookie Of The Year_rk: The player's ranking or points in the voting for the Rookie of the Year award.

Sixth Man Of The Year_rk: The player's ranking or points in the voting for the Sixth Man of the Year award.

all_nba_points_rk: The player's ranking or points in the voting for the All-NBA teams.

all_rookie_points_rk: The player's ranking or points in the voting for the All-Rookie teams.

In [355]:
awards

,season,nbapersonid,All NBA Defensive First Team,All NBA Defensive Second Team,All NBA First Team,All NBA Second Team,All NBA Third Team,All Rookie First Team,All Rookie Second Team,Bill Russell NBA Finals MVP,...,all_star_game,rookie_all_star_game,allstar_rk,Defensive Player Of The Year_rk,Most Improved Player_rk,Most Valuable Player_rk,Rookie Of The Year_rk,Sixth Man Of The Year_rk,all_nba_points_rk,all_rookie_points_rk
0,2007,708.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,True,False,1.0,1.0,NaN,3.0,NaN,NaN,NaN,NaN
1,2007,947.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,True,False,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2007,948.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,3.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN
3,2007,959.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,True,False,4.0,NaN,NaN,9.0,NaN,NaN,NaN,NaN
4,2007,977.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,True,False,1.0,5.0,NaN,1.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4324,2015,1626170.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,24.0
4325,2015,1626202.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,24.0
4326,2015,1626273.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,24.0
4327,2018,1628971.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18.0


let's inspect the dataset

In [356]:
print(awards.isnull().sum())
awards.describe()

season                                0
nbapersonid                           8
All NBA Defensive First Team       3636
All NBA Defensive Second Team      3636
All NBA First Team                 3636
All NBA Second Team                3636
All NBA Third Team                 3636
All Rookie First Team              3636
All Rookie Second Team             3636
Bill Russell NBA Finals MVP        3636
Player Of The Month                3636
Player Of The Week                 3636
Rookie Of The Month                3636
all_star_game                      3645
rookie_all_star_game               3645
allstar_rk                          638
Defensive Player Of The Year_rk    4074
Most Improved Player_rk            3929
Most Valuable Player_rk            4127
Rookie Of The Year_rk              4206
Sixth Man Of The Year_rk           4092
all_nba_points_rk                  3935
all_rookie_points_rk               4063
dtype: int64


,season,nbapersonid,All NBA Defensive First Team,All NBA Defensive Second Team,All NBA First Team,All NBA Second Team,All NBA Third Team,All Rookie First Team,All Rookie Second Team,Bill Russell NBA Finals MVP,...,Player Of The Week,Rookie Of The Month,allstar_rk,Defensive Player Of The Year_rk,Most Improved Player_rk,Most Valuable Player_rk,Rookie Of The Year_rk,Sixth Man Of The Year_rk,all_nba_points_rk,all_rookie_points_rk
count,4329.000000,4.321000e+03,693.000000,693.000000,693.000000,693.000000,693.000000,693.000000,693.000000,693.000000,...,693.000000,693.000000,3691.000000,255.000000,400.000000,202.000000,123.000000,237.000000,394.000000,266.000000
mean,2016.687688,1.622733e+06,0.109668,0.108225,0.108225,0.108225,0.108225,0.111111,0.109668,0.021645,...,0.940837,0.233766,58.173124,9.258824,13.540000,7.207921,4.853659,8.177215,18.390863,12.409774
std,3.781453,4.221668e+07,0.312701,0.310889,0.310889,0.310889,0.310889,0.314497,0.312701,0.145627,...,1.175727,0.790231,40.466750,5.409571,7.675329,3.915315,2.804221,4.468608,10.581058,7.031019
min,2007.000000,2.550000e+02,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
25%,2015.000000,2.015650e+05,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,20.000000,5.000000,7.000000,4.000000,3.000000,4.000000,9.000000,6.250000
50%,2018.000000,2.034710e+05,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,1.000000,0.000000,56.000000,9.000000,13.000000,7.000000,5.000000,8.000000,18.000000,13.000000
75%,2020.000000,1.627885e+06,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,1.000000,0.000000,92.000000,13.000000,20.000000,10.000000,7.000000,11.000000,27.000000,18.000000
max,2021.000000,1.962937e+09,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,7.000000,6.000000,157.000000,25.000000,30.000000,17.000000,13.000000,18.000000,41.000000,26.000000


Based on the variable in awards file, we believed that the absence of awards for certain team members can be just as informative as the presence of awards for others, offering a more nuanced understanding of what contributes to overall team success. So we decided to keep them. 

In [357]:
null_values_summary2 = awards.isnull().sum()
columns_with_nulls2 = null_values_summary2[null_values_summary2 > 0]
print(columns_with_nulls2)

nbapersonid                           8
All NBA Defensive First Team       3636
All NBA Defensive Second Team      3636
All NBA First Team                 3636
All NBA Second Team                3636
All NBA Third Team                 3636
All Rookie First Team              3636
All Rookie Second Team             3636
Bill Russell NBA Finals MVP        3636
Player Of The Month                3636
Player Of The Week                 3636
Rookie Of The Month                3636
all_star_game                      3645
rookie_all_star_game               3645
allstar_rk                          638
Defensive Player Of The Year_rk    4074
Most Improved Player_rk            3929
Most Valuable Player_rk            4127
Rookie Of The Year_rk              4206
Sixth Man Of The Year_rk           4092
all_nba_points_rk                  3935
all_rookie_points_rk               4063
dtype: int64


let's look for some specific columns where the players' id is the same. We then noticed that all of them have no other data. We assume that they are less relevent to our analysis later so we can drop them off.

In [358]:
null_nbapersonid_rows = awards[awards['nbapersonid'].isnull()]
null_nbapersonid_rows


,season,nbapersonid,All NBA Defensive First Team,All NBA Defensive Second Team,All NBA First Team,All NBA Second Team,All NBA Third Team,All Rookie First Team,All Rookie Second Team,Bill Russell NBA Finals MVP,...,all_star_game,rookie_all_star_game,allstar_rk,Defensive Player Of The Year_rk,Most Improved Player_rk,Most Valuable Player_rk,Rookie Of The Year_rk,Sixth Man Of The Year_rk,all_nba_points_rk,all_rookie_points_rk
1479,2021,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,42.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1480,2021,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,109.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1481,2021,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,114.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1931,2020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,61.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1932,2020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,62.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2402,2019,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,66.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2876,2018,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,44.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3388,2017,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,90.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [359]:
awards_clean = awards.dropna(subset=['nbapersonid'])


Detecting if there are some duplicate rows.

In [360]:
duplicates2 = awards_clean.duplicated().sum()
print(f"Number of duplicate rows: {duplicates2}")

Number of duplicate rows: 9


In [361]:
duplicated_rows = awards_clean[awards_clean.duplicated()]
duplicated_rows

,season,nbapersonid,All NBA Defensive First Team,All NBA Defensive Second Team,All NBA First Team,All NBA Second Team,All NBA Third Team,All Rookie First Team,All Rookie Second Team,Bill Russell NBA Finals MVP,...,all_star_game,rookie_all_star_game,allstar_rk,Defensive Player Of The Year_rk,Most Improved Player_rk,Most Valuable Player_rk,Rookie Of The Year_rk,Sixth Man Of The Year_rk,all_nba_points_rk,all_rookie_points_rk
2867,2018,203710.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,101.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2870,2018,202498.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,90.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2871,2018,202969.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,90.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3384,2017,203710.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,91.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3385,2017,202498.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,68.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3386,2017,202969.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,68.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3801,2016,202498.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,89.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3802,2016,202969.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,89.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3804,2016,2034.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,75.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


we found that these players also has less useful dataset so that we assume that they are also some not useful players.For all of the nba players , they did not receive any rewards as marked as null. We will not consider them as individual, but they are still useful for team performance.

In [362]:
not_useful_players = duplicated_rows['nbapersonid']
not_useful_players

2867    203710.0
2870    202498.0
2871    202969.0
3384    203710.0
3385    202498.0
3386    202969.0
3801    202498.0
3802    202969.0
3804      2034.0
Name: nbapersonid, dtype: float64

In [363]:
print(awards_clean.dtypes)

season                               int64
nbapersonid                        float64
All NBA Defensive First Team       float64
All NBA Defensive Second Team      float64
All NBA First Team                 float64
All NBA Second Team                float64
All NBA Third Team                 float64
All Rookie First Team              float64
All Rookie Second Team             float64
Bill Russell NBA Finals MVP        float64
Player Of The Month                float64
Player Of The Week                 float64
Rookie Of The Month                float64
all_star_game                       object
rookie_all_star_game                object
allstar_rk                         float64
Defensive Player Of The Year_rk    float64
Most Improved Player_rk            float64
Most Valuable Player_rk            float64
Rookie Of The Year_rk              float64
Sixth Man Of The Year_rk           float64
all_nba_points_rk                  float64
all_rookie_points_rk               float64
dtype: obje

we found that all_star_game and rookie_all_star_game is object but we need them to be consistent for latter analysis. So we first convert all null to 0, which means the null value representing the no record.

In [364]:
award_columns = ['All NBA Defensive First Team', 'All NBA Defensive Second Team', 
                 'All NBA First Team', 'All NBA Second Team', 'All NBA Third Team', 
                 'All Rookie First Team', 'All Rookie Second Team', 
                 'Bill Russell NBA Finals MVP', 'Player Of The Month', 
                 'Player Of The Week', 'Rookie Of The Month', 'all_star_game', 
                 'rookie_all_star_game', 'allstar_rk', 'Defensive Player Of The Year_rk', 
                 'Most Improved Player_rk', 'Most Valuable Player_rk', 
                 'Rookie Of The Year_rk', 'Sixth Man Of The Year_rk', 
                 'all_nba_points_rk', 'all_rookie_points_rk']

for col in award_columns:
    awards_clean[col].fillna(0, inplace=True)

In [365]:
print(awards_clean.isnull().sum())

season                             0
nbapersonid                        0
All NBA Defensive First Team       0
All NBA Defensive Second Team      0
All NBA First Team                 0
All NBA Second Team                0
All NBA Third Team                 0
All Rookie First Team              0
All Rookie Second Team             0
Bill Russell NBA Finals MVP        0
Player Of The Month                0
Player Of The Week                 0
Rookie Of The Month                0
all_star_game                      0
rookie_all_star_game               0
allstar_rk                         0
Defensive Player Of The Year_rk    0
Most Improved Player_rk            0
Most Valuable Player_rk            0
Rookie Of The Year_rk              0
Sixth Man Of The Year_rk           0
all_nba_points_rk                  0
all_rookie_points_rk               0
dtype: int64


Then we mark all existed object type to 1 and 0 of floating type.

In [366]:
awards_clean['all_star_game'] = awards_clean['all_star_game'].map({False: 0, True: 1}).astype(float)
awards_clean['rookie_all_star_game'] = awards_clean['rookie_all_star_game'].map({False: 0, True: 1}).astype(float)

Convert some features name to a more readable form

In [367]:

awards_clean = awards_clean.rename(columns={'nbapersonid': 'personID'})
awards_clean = awards_clean.rename(columns={'all_star_game': 'all_star?'})
awards_clean = awards_clean.rename(columns={'rookie_all_star_game': 'rookie_all_star?'})
awards_clean

,season,personID,All NBA Defensive First Team,All NBA Defensive Second Team,All NBA First Team,All NBA Second Team,All NBA Third Team,All Rookie First Team,All Rookie Second Team,Bill Russell NBA Finals MVP,...,all_star?,rookie_all_star?,allstar_rk,Defensive Player Of The Year_rk,Most Improved Player_rk,Most Valuable Player_rk,Rookie Of The Year_rk,Sixth Man Of The Year_rk,all_nba_points_rk,all_rookie_points_rk
0,2007,708.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,1.0,1.0,0.0,3.0,0.0,0.0,0.0,0.0
1,2007,947.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2007,948.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,3.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2007,959.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,4.0,0.0,0.0,9.0,0.0,0.0,0.0,0.0
4,2007,977.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,1.0,5.0,0.0,1.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4324,2015,1626170.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,24.0
4325,2015,1626202.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,24.0
4326,2015,1626273.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,24.0
4327,2018,1628971.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,18.0


In [368]:
awards_clean.head()

,season,personID,All NBA Defensive First Team,All NBA Defensive Second Team,All NBA First Team,All NBA Second Team,All NBA Third Team,All Rookie First Team,All Rookie Second Team,Bill Russell NBA Finals MVP,...,all_star?,rookie_all_star?,allstar_rk,Defensive Player Of The Year_rk,Most Improved Player_rk,Most Valuable Player_rk,Rookie Of The Year_rk,Sixth Man Of The Year_rk,all_nba_points_rk,all_rookie_points_rk
0,2007,708.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,1.0,1.0,0.0,3.0,0.0,0.0,0.0,0.0
1,2007,947.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2007,948.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,3.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2007,959.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,4.0,0.0,0.0,9.0,0.0,0.0,0.0,0.0
4,2007,977.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,1.0,5.0,0.0,1.0,0.0,0.0,0.0,0.0


## team_data part

In [369]:
team_data.columns

Index(['nbateamid', 'team', 'season', 'games', 'off_rtg', 'def_rtg', 'net_rtg',
       'W', 'L'],
      dtype='object')

nbateamid: This is a unique identifier for each NBA team.

team: The name of the NBA team.

season: The NBA season for which the statistics are recorded.

games: The total number of games the team played during the specified season.

off_rtg: Offensive rating. This is an estimate of a team's offensive efficiency and is typically calculated as the number of points scored per 100 possessions by the team.

def_rtg: Defensive rating. This measures a team's defensive efficiency and is usually the number of points allowed per 100 possessions by the team.

net_rtg: Net rating. This is calculated by subtracting the defensive rating from the offensive rating. A positive net rating indicates that a team scores more points per 100 possessions than it allows, suggesting overall effectiveness.

W: The number of wins a team achieved in the season.

L: The number of losses a team suffered in the season.

In [370]:
team_data.head()

,nbateamid,team,season,games,off_rtg,def_rtg,net_rtg,W,L
0,1610612737,ATL,2007,82,106.9,108.9,-2.0,37,45
1,1610612751,BKN,2007,82,104.0,109.4,-5.4,34,48
2,1610612738,BOS,2007,82,110.2,98.9,11.3,66,16
3,1610612766,CHA,2007,82,104.6,109.4,-4.8,32,50
4,1610612741,CHI,2007,82,103.9,107.2,-3.3,33,49


let's inspect the data

In [371]:
print(team_data.dtypes)

nbateamid      int64
team          object
season         int64
games          int64
off_rtg      float64
def_rtg      float64
net_rtg      float64
W              int64
L              int64
dtype: object


In [372]:
print(team_data.isnull().sum())
team_data.describe()

nbateamid    0
team         0
season       0
games        0
off_rtg      0
def_rtg      0
net_rtg      0
W            0
L            0
dtype: int64


,nbateamid,season,games,off_rtg,def_rtg,net_rtg,W,L
count,4.500000e+02,450.000000,450.000000,450.000000,450.000000,450.000000,450.000000,450.000000
mean,1.610613e+09,2014.000000,79.502222,108.156000,108.156889,-0.000889,39.751111,39.751111
std,8.665075e+00,4.325302,5.190041,3.966808,3.679923,4.955616,12.572119,12.445348
min,1.610613e+09,2007.000000,64.000000,95.200000,98.200000,-15.200000,7.000000,9.000000
25%,1.610613e+09,2010.000000,82.000000,105.300000,105.600000,-3.500000,30.250000,30.000000
50%,1.610613e+09,2014.000000,82.000000,108.100000,108.200000,0.200000,41.000000,39.000000
75%,1.610613e+09,2018.000000,82.000000,111.100000,110.775000,3.600000,49.000000,49.000000
max,1.610613e+09,2021.000000,82.000000,118.300000,117.600000,11.600000,73.000000,72.000000


In [373]:
duplicate3 = team_data.duplicated().sum()
print(f"Number of duplicate rows: {duplicate3}")

Number of duplicate rows: 0


To add two more feature of winpercentage and lose percentage

In [374]:
team_data_clean = team_data
team_data_clean['win_percentage'] = team_data_clean['W'] /team_data_clean['games'] 
team_data_clean['lose_percentage'] = team_data_clean['L'] /team_data_clean['games'] 
team_data_clean['win_percentage'] = team_data_clean['win_percentage'].round(2)
team_data_clean['lose_percentage'] = team_data_clean['lose_percentage'].round(2)
team_data_clean.head()

,nbateamid,team,season,games,off_rtg,def_rtg,net_rtg,W,L,win_percentage,lose_percentage
0,1610612737,ATL,2007,82,106.9,108.9,-2.0,37,45,0.45,0.55
1,1610612751,BKN,2007,82,104.0,109.4,-5.4,34,48,0.41,0.59
2,1610612738,BOS,2007,82,110.2,98.9,11.3,66,16,0.80,0.20
3,1610612766,CHA,2007,82,104.6,109.4,-4.8,32,50,0.39,0.61
4,1610612741,CHI,2007,82,103.9,107.2,-3.3,33,49,0.40,0.60


Change some features' name to increase readability

In [375]:
team_data_clean = team_data.rename(columns={'nbateamid': 'teamID'})
team_data_clean = team_data.rename(columns={'team': 'team_name'})
team_data_clean = team_data.rename(columns={'games': 'total_game'})
team_data_clean = team_data.rename(columns={'off_rtg': 'offensive_rate'})
team_data_clean = team_data.rename(columns={'def_rtg': 'defensive_rate'})
team_data_clean = team_data.rename(columns={'net_rtg	': 'net_rate'})
team_data_clean

,nbateamid,team,season,games,off_rtg,def_rtg,net_rtg,W,L,win_percentage,lose_percentage
0,1610612737,ATL,2007,82,106.9,108.9,-2.0,37,45,0.45,0.55
1,1610612751,BKN,2007,82,104.0,109.4,-5.4,34,48,0.41,0.59
2,1610612738,BOS,2007,82,110.2,98.9,11.3,66,16,0.80,0.20
3,1610612766,CHA,2007,82,104.6,109.4,-4.8,32,50,0.39,0.61
4,1610612741,CHI,2007,82,103.9,107.2,-3.3,33,49,0.40,0.60
...,...,...,...,...,...,...,...,...,...,...,...
445,1610612758,SAC,2021,82,109.9,115.3,-5.4,30,52,0.37,0.63
446,1610612759,SAS,2021,82,112.4,112.3,0.1,34,48,0.41,0.59
447,1610612761,TOR,2021,82,112.9,110.5,2.4,48,34,0.59,0.41
448,1610612762,UTA,2021,82,116.7,110.5,6.2,49,33,0.60,0.40


## rebounding data part

In [376]:
rebounding_data.columns

Index(['team', 'opp_team', 'gamedate', 'game_number', 'offensive_rebounds',
       'off_rebound_chances', 'oreb_pct'],
      dtype='object')

In [377]:
rebounding_data.head()

,team,opp_team,gamedate,game_number,offensive_rebounds,off_rebound_chances,oreb_pct
0,BOS,PHI,2022-10-18,1,10,39,0.256410
1,PHI,BOS,2022-10-18,1,8,42,0.190476
2,GSW,LAL,2022-10-18,1,16,57,0.280702
3,LAL,GSW,2022-10-18,1,14,57,0.245614
4,ORL,DET,2022-10-19,1,13,47,0.276596


team: This is the name of the NBA team

opp_team: The opposing team that team was playing against

gamedate: The date on which the game was played.

game_number: This represents the chronological order of the game within a season for team. For instance, the first game of the season would be 1, the second game would be 2, and so on.

offensive_rebounds: The total number of rebounds that team collected on the offensive end of the court. Offensive rebounds are grabbed after an offensive shot attempt has been missed, giving the team another opportunity to score within the same possession.

off_rebound_chances: The number of opportunities team had to grab an offensive rebound. This could be the total number of missed offensive shots that could have been rebounded.

oreb_pct: Offensive rebound percentage. This is a measure of the proportion of available offensive rebounds that team actually secured. It is calculated as the number of offensive rebounds divided by the total number of offensive rebounding opportunities (the sum of team's offensive rebounds and the opp_team's defensive rebounds). This statistic provides insight into how effective a team is at retrieving their own missed shots.

let's inspect the data

In [378]:
rebounding_data.dtypes

team                    object
opp_team                object
gamedate                object
game_number              int64
offensive_rebounds       int64
off_rebound_chances      int64
oreb_pct               float64
dtype: object

In [379]:
print(rebounding_data.isnull().sum())
rebounding_data.describe()

team                   0
opp_team               0
gamedate               0
game_number            0
offensive_rebounds     0
off_rebound_chances    0
oreb_pct               0
dtype: int64


,game_number,offensive_rebounds,off_rebound_chances,oreb_pct
count,2460.000000,2460.000000,2460.000000,2460.000000
mean,41.500000,13.741057,48.921138,0.277861
std,23.674413,4.643713,7.142900,0.075100
min,1.000000,1.000000,28.000000,0.033333
25%,21.000000,10.000000,44.000000,0.225000
50%,41.500000,13.000000,49.000000,0.277778
75%,62.000000,17.000000,53.000000,0.333333
max,82.000000,30.000000,81.000000,0.547170


In [380]:
duplicates4 = rebounding_data.duplicated().sum()
print(f"Number of duplicate rows: {duplicates4}")

Number of duplicate rows: 0


In [381]:
rebounding_data['gamedate'].unique()

array(['2022-10-18', '2022-10-19', '2022-10-20', '2022-10-21',
       '2022-10-22', '2022-10-23', '2022-10-24', '2022-10-25',
       '2022-10-26', '2022-10-27', '2022-10-28', '2022-10-29',
       '2022-10-30', '2022-10-31', '2022-11-01', '2022-11-02',
       '2022-11-03', '2022-11-04', '2022-11-05', '2022-11-06',
       '2022-11-07', '2022-11-09', '2022-11-10', '2022-11-11',
       '2022-11-12', '2022-11-13', '2022-11-14', '2022-11-15',
       '2022-11-16', '2022-11-17', '2022-11-18', '2022-11-19',
       '2022-11-20', '2022-11-21', '2022-11-22', '2022-11-23',
       '2022-11-25', '2022-11-26', '2022-11-27', '2022-11-28',
       '2022-11-29', '2022-11-30', '2022-12-01', '2022-12-02',
       '2022-12-03', '2022-12-04', '2022-12-05', '2022-12-06',
       '2022-12-07', '2022-12-08', '2022-12-09', '2022-12-10',
       '2022-12-11', '2022-12-12', '2022-12-13', '2022-12-14',
       '2022-12-15', '2022-12-16', '2022-12-17', '2022-12-18',
       '2022-12-19', '2022-12-20', '2022-12-21', '2022-

Based on this, we know that the rebouding data start from 2022-10-18

In [382]:
rebounding_data

,team,opp_team,gamedate,game_number,offensive_rebounds,off_rebound_chances,oreb_pct
0,BOS,PHI,2022-10-18,1,10,39,0.256410
1,PHI,BOS,2022-10-18,1,8,42,0.190476
2,GSW,LAL,2022-10-18,1,16,57,0.280702
3,LAL,GSW,2022-10-18,1,14,57,0.245614
4,ORL,DET,2022-10-19,1,13,47,0.276596
...,...,...,...,...,...,...,...
2455,LAC,PHX,2023-04-09,82,18,56,0.321429
2456,MEM,OKC,2023-04-09,82,12,55,0.218182
2457,POR,GSW,2023-04-09,82,11,61,0.180328
2458,SAC,DEN,2023-04-09,82,12,50,0.240000


convert the features name to make them more readable

In [383]:
rebounding_data_cleam = rebounding_data
rebounding_data_cleam = rebounding_data.rename(columns={'opp_team': 'opposite_team_name'})
rebounding_data_cleam = rebounding_data.rename(columns={'team': 'team_name'})
rebounding_data_cleam = rebounding_data.rename(columns={'oreb_pct': 'Offensive_rebound_perc'})
rebounding_data_cleam

,team,opp_team,gamedate,game_number,offensive_rebounds,off_rebound_chances,Offensive_rebound_perc
0,BOS,PHI,2022-10-18,1,10,39,0.256410
1,PHI,BOS,2022-10-18,1,8,42,0.190476
2,GSW,LAL,2022-10-18,1,16,57,0.280702
3,LAL,GSW,2022-10-18,1,14,57,0.245614
4,ORL,DET,2022-10-19,1,13,47,0.276596
...,...,...,...,...,...,...,...
2455,LAC,PHX,2023-04-09,82,18,56,0.321429
2456,MEM,OKC,2023-04-09,82,12,55,0.218182
2457,POR,GSW,2023-04-09,82,11,61,0.180328
2458,SAC,DEN,2023-04-09,82,12,50,0.240000


# visualization

In [384]:
num_cols = player_data_clean.select_dtypes(include=['float64', 'int64'])
columns_to_exclude = ['nbapersonid', 'nbateamid']
num_cols = num_cols.drop(columns=columns_to_exclude)
correlation_matrix = num_cols.corr()
plt.figure(figsize=(20, 15))

mask = np.triu(np.ones_like(correlation_matrix, dtype=bool))
sns.heatmap(correlation_matrix, mask=mask, annot=True, fmt=".2f", cmap='coolwarm', 
            square=True, annot_kws={'size': 5})
plt.xticks(rotation=90)
plt.yticks(rotation=0)
plt.show()

KeyError: "['nbapersonid', 'nbateamid'] not found in axis"

In [ ]:
num_plots = len(num_cols)
num_rows = (num_plots + 1) // 2 # Adjust 2 to change how many plots per row

# Set the figure size (width, height)
plt.figure(figsize=(15, num_rows * 5))

# Create subplots in a grid
for i, col in enumerate(num_cols):
    plt.subplot(num_rows, 3, i + 1)  # Change 2 to the number of columns you want
    sns.histplot(player_data_clean[col], kde=True)
    plt.title(f'Distribution of {col}')
    plt.xlabel(col)
    plt.ylabel('Frequency')

# Adjust layout to prevent overlap
plt.tight_layout()
plt.show()

ValueError: Image size of 1500x2123000 pixels is too large. It must be less than 2^16 in each direction.

Error in callback <function _draw_all_if_interactive at 0x0000029172105300> (for post_execute):


ValueError: Image size of 1500x2123000 pixels is too large. It must be less than 2^16 in each direction.

ValueError: Image size of 1500x2123000 pixels is too large. It must be less than 2^16 in each direction.

<Figure size 1500x2.123e+06 with 44 Axes>